In [ ]:
import os
import time

import math
import numpy as np 
import pandas as pd
import seaborn as sns; sns.set(style="ticks", color_codes=True)

from sklearn.metrics import mean_absolute_error as MAE, mean_squared_error as MSE
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import RFE

import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
mpl.rcParams['figure.figsize'] = (20, 13)
mpl.rcParams['axes.grid'] = False

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 256)

from pandas.core.common import SettingWithCopyWarning

import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# **Data Loading**

In [ ]:
from dateutil.parser import parse

date_parser = lambda date: parse(date)

In [ ]:
df = pd.read_csv('../input/sunspots/Sunspots.csv',
                 usecols=['Date', 'Monthly Mean Total Sunspot Number'], 
                 parse_dates=['Date'],
                 date_parser=date_parser)
df.rename(columns={'Monthly Mean Total Sunspot Number': 'Monthly_Average_Sunspot'}, inplace=True)
df.set_index(keys='Date', drop=True, inplace=True)

df.head()

## Add **seasonality** by **Sin-Cos Extraction**

In [ ]:
date_time = pd.Series(df.index)
date_time.head()

In [ ]:
from datetime import date, datetime

timestamp_dt = date_time.map(datetime.timestamp)
DAY = 24*60*60
MONTH = 30*DAY
YEAR = 12*MONTH
DECADE = 10*YEAR
PERIOD = 11*YEAR
CENTURY = 10*DECADE 

df['Month_sin'] = np.sin(timestamp_dt * (2 * np.pi / MONTH)).values
df['Month_cos'] = np.cos(timestamp_dt * (2 * np.pi / MONTH)).values
df['Year_sin'] = np.sin(timestamp_dt * (2 * np.pi / YEAR)).values
df['Year_cos'] = np.cos(timestamp_dt * (2 * np.pi / YEAR)).values
df['Decade_sin'] = np.sin(timestamp_dt * (2 * np.pi / DECADE)).values
df['Decade_cos'] = np.cos(timestamp_dt * (2 * np.pi / DECADE)).values
# df['Period_sin'] = np.sin(timestamp_dt * (2 * np.pi / PERIOD)).values
# df['Period_cos'] = np.cos(timestamp_dt * (2 * np.pi / PERIOD)).values
df['Century_sin'] = np.sin(timestamp_dt * (2 * np.pi / CENTURY)).values
df['Century_cos'] = np.cos(timestamp_dt * (2 * np.pi / CENTURY)).values

for col in ['Month', 'Year', 'Decade', 'Century']:
    if col == 'Month':
        N_STEPS = 13
    elif col == 'Year':
        N_STEPS = 25
    elif col == 'Decade':
        N_STEPS = 200
    elif col == 'Century':
        N_STEPS = 1500
    
    plt.plot(date_time.values[...,:N_STEPS], df[col+'_sin'].values[...,:N_STEPS], 'ro', 
             date_time.values[...,:N_STEPS], df[col+'_cos'].values[...,:N_STEPS], 'bo')
    plt.show()

In [ ]:
N_YEARS = 100
SEQ_LEN = 12
train_df = df[-3*N_YEARS*12:-N_YEARS*12]
test_df = df[-N_YEARS*12:]

train_size, test_size = len(train_df), len(test_df)
print(train_size, test_size)

plt.plot(train_df.index, train_df.Monthly_Average_Sunspot, 'bo',
         test_df.index, test_df.Monthly_Average_Sunspot, 'ro')

## **Build dataset**

In [ ]:
datasets = dict()

datasets['train'] = train_df
datasets['test'] = pd.concat([train_df[-SEQ_LEN+1:], test_df])

In [ ]:
datasets['train'].describe()

In [ ]:
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()
data_generators = dict()
for ds_name, ds in datasets.items():
    data = ds.values
    X, y = data[:, 1:].astype(np.float32), data[:, 0].astype(np.float32)
    if ds_name == 'train':
        y = scaler.fit_transform(y.reshape(-1, 1)).flatten()
    else:
        y = scaler.transform(y.reshape(-1, 1)).flatten()
    print(f"{ds_name}: {data.shape} --> {X.shape} + {y.shape}")
    data_generators[ds_name] = timeseries_dataset_from_array(X, y,
                                                             sequence_length=SEQ_LEN, 
                                                             sequence_stride=1,
                                                             sampling_rate=1,
                                                             batch_size=64,)
    for batch in data_generators[ds_name].take(1):
        inputs, targets = batch
        # print(targets)
        print("\t Input shape:", inputs.numpy().shape)
        print("\t Target shape:", targets.numpy().shape)

In [ ]:
data_batch = dict()
for ds_name, generator in data_generators.items():
    X_all, y_all = [], []
    for X_batch, y_batch in generator:
        X_all.extend(X_batch.numpy().tolist())
        y_all.extend(y_batch.numpy().tolist())
    X_all = np.array(X_all).astype(np.float32)
    y_all = np.array(y_all).astype(np.float32)
    
    # TSAI Input Shape: (N_samples, N_features, Max_seq_len)
    X_all = np.transpose(X_all, axes=(0,2,1))
    
    data_batch[ds_name] = [X_all, y_all]
    print(ds_name, X_all.shape, y_all.shape)

In [ ]:
X_train, y_train = data_batch['train']
X_test, y_test = data_batch['test']

# **Modelling**

## **Loss Function**

In [ ]:
# Huber Loss, aka Smoothed Mean Absolute Error
from tensorflow.keras.losses import Huber, Reduction

loss_func = Huber(reduction=Reduction.NONE)

In [ ]:
loss_df = pd.DataFrame()
loss_df['Date'] = test_df.index

## **TSAI**

In [ ]:
!pip install --ignore-installed tsai

In [ ]:
from tsai.all import *

import torch

def torch2np(tensor: torch.Tensor) -> np.array:
    if torch.cuda.is_available():
        tensor = tensor.cpu()
    return tensor.numpy()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
scorer = make_scorer(MSE, greater_is_better=False)

## **MiniRocket**

In [ ]:
params = {'num_features': 10_000, 
          'max_dilations_per_kernel': 16, 
          'normalize_features': False, 
          'verbose': True, 
          'scoring': scorer}
for n_estimators in [1,3,5,7]:
    if n_estimators == 1:
        model = MiniRocketRegressor(**params)
    else:
        model = MiniRocketVotingRegressor(n_estimators=n_estimators, **params)

    print(f"\n\n Training MiniRocket-{n_estimators} ...")
    timer.start(False)
    model.fit(X_train, y_train)
    t = timer.stop()
    print(f"\t ... in {t}")
    
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions.reshape((-1,1)))
    
    plt.plot(train_df.index, train_df.Monthly_Average_Sunspot, 'ro',
             test_df.index, test_df.Monthly_Average_Sunspot, 'yo', 
             test_df.index, predictions, 'bo')
    plt.show()
    
    loss = loss_func(test_df.Monthly_Average_Sunspot.values.reshape(-1,1),
                     predictions).numpy()
    loss_df[f'MiniRocket-{n_estimators}'] = loss.flatten()
    
    del model
    
loss_df.describe()

## **InceptionTime / TSTransformer**

In [ ]:
DL_models = {
    "ResNet": (ResNetPlus, {'nf': 32, 'ks': [7, 5, 3], 'seq_len': SEQ_LEN}), 
    "XceptionTime": (XceptionTime, {'nf': 16, 'adaptive_size': 32, 'residual': True}), 
    "XceptionTimePlus": (XceptionTimePlus, {'nf': 16, 'adaptive_size': 32, 'residual': True}), 
    "InceptionTime": (InceptionTime, {'nf': 32, 'ks': SEQ_LEN}), 
    "InceptionTimePlus": (InceptionTimePlus, {'nf': 32, 'ks': SEQ_LEN, 'bottleneck': True, 'depth': 6, 'dilation': 1, 'stride': 1}), 
    "TSTransformer": (TST, {'max_seq_len': SEQ_LEN, 'd_model': 32, 'd_ff': 16, 'n_layers': 2, 'n_heads': 4, }), 
    "TSTransformerPlus": (TSTPlus, {'max_seq_len': SEQ_LEN, 'd_model': 32, 'd_ff': 16, 'n_layers': 2, 'n_heads': 4, }), 
}

In [ ]:
X_dl, y_dl, splits = combine_split_data([X_train, X_test], [y_train, y_test])

transformations = [None, [TSRegression()]]
batch_transformations = [TSStandardize(by_sample=False, by_var=False)]
dsets = TSDatasets(X_dl, y_dl, splits=splits, tfms=transformations, inplace=True)
dloaders = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[64, 32], batch_tfms=batch_transformations, num_workers=0)

In [ ]:
for model_name, (model, params) in DL_models.items():
    
    print(f"\n\n Training {model_name} ...")
    timer.start(False)
    
    # Create model
    model = create_model(model, dls=dloaders, **params)
    learner = Learner(dls=dloaders, model=model, metrics=[mae, rmse], opt_func=Adam)
    
    # Find best learning-rate
    lr_lowest, lr_steepest = learner.lr_find(start_lr=1e-7, end_lr=1e0, num_it=169)
    print(f"\t ... with learning-rate = {lr_lowest}")
    
    # Train
    learner.fit_one_cycle(n_epoch=50, lr_max=lr_lowest)
    t = timer.stop()
    print(f"\t ... in {t}")
    
    # Evaluate
    X_test = torch.Tensor(X_test).to(device)
    y_pred = learner.get_X_preds(X_test)[0]
    y_pred = torch2np(y_pred.detach())
    
    predictions = scaler.inverse_transform(y_pred.reshape((-1,1)))
    
    # Visualize
    plt.show()
    plt.plot(train_df.index, train_df.Monthly_Average_Sunspot, 'ro',
             test_df.index, test_df.Monthly_Average_Sunspot, 'yo', 
             test_df.index, predictions, 'bo')
    plt.show()
    
    # Loss statistics
    loss = loss_func(test_df.Monthly_Average_Sunspot.values.reshape(-1,1),
                     predictions).numpy()
    loss_df[f'{model_name}'] = loss.flatten()
    
    del model, learner

In [ ]:
loss_df.describe()